In [1]:
from dotenv import load_dotenv
import os
from glob import glob

from pprint import pprint
import json

In [6]:
#임베딩이 있어야 db를 만들 수 있다.
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

c:\Users\KOSALOVE\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-I6Y7WdhY-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_sample_collection",
    persist_directory="./chroma_db",
    embedding_function=embeddings_model
)

In [ ]:
chroma_db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [11]:
from langchain_core.documents import Document

documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

doc_objects = []
for i, content in enumerate(documents):
    doc = Document(
        page_content=content,
        metadata={"source": "AI_Textbook", "chapter": f"Chapter {i}"},
    )

    doc_objects.append(doc)

# 순차적 id 리스트 생성
doc_ids = [f"doc_{i}" for i in range(1, len(doc_objects)+1)]

added_docs_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)

#이전과의 차이점
#각 문서를 doc 객체로 해서 chroma db에 넣는다.
#각각의 ids를 넣음.

print(f"{len(added_docs_ids)} 개의 문서가 성공적으로 DB에 추가되었습니다.")
print(added_docs_ids)

5 개의 문서가 성공적으로 DB에 추가되었습니다.
['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5']


In [14]:
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"질문: {query}")
for doc in results :
    print(f"문서 ID: {doc.metadata['chapter']}, 내용: {doc.page_content}")

질문: 인공지능과 머신러닝의 관계는?
문서 ID: Chapter 1, 내용: 머신러닝은 인공지능의 하위 분야입니다.
문서 ID: Chapter 2, 내용: 딥러닝은 머신러닝의 한 종류입니다.


In [16]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 0', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 1', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [25]:
# chroma_db.update_documents(document_ids=added_docs_ids, documents=doc_objects)
# Document(page_content, metadata={}) # doc 객체 안에 page_content 값이 있어야한다
update_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 한 분야로 머신러닝과 딥러닝을 포함합니다.",
    metadata={"source": "AI_Textbook", "chapter": "Chapter 6"}
)

chroma_db.update_document(document_id="doc_1", document=update_document_1)

print("문서 업데이트 완료")

문서 업데이트 완료


In [ ]:
chroma_db.get()

#doc 1이 '인공지능은 컴퓨터 과학의 한 분야로 머신러닝과 딥러닝을 포함합니다.'로 변경됨

{'ids': ['doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_5',
  'doc_ex_1',
  'doc_ex_2',
  'doc_ex_3',
  'doc_ex_4',
  'doc_ex_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로 머신러닝과 딥러닝을 포함합니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.',
  '스트레칭은 운동 전후에 근육을 이완시키는 데 도움이 됩니다.',
  '유산소 운동은 심혈관 건강에 좋습니다.',
  '근력 운동은 근육을 강화하고 기초 대사량을 높입니다.',
  '요가는 유연성을 향상시키고 정신적인 안정감을 줍니다.',
  '사무실에서 할 수 있는 간단한 목 스트레칭이 있습니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 1', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'},
  {'section': 'Section 0', 'source': 'Exercise_Guide'},
  {'section': 'Section 1', 'source': 'Exercise_Guide'},
  {'section': 'Section 2', 'source': 'Exercise_Guide'}

In [27]:
chroma_db.delete(ids=["doc_5"])

In [29]:
chroma_db.get()
#doc_5번 삭제됨

{'ids': ['doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_ex_1',
  'doc_ex_2',
  'doc_ex_3',
  'doc_ex_4',
  'doc_ex_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로 머신러닝과 딥러닝을 포함합니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '스트레칭은 운동 전후에 근육을 이완시키는 데 도움이 됩니다.',
  '유산소 운동은 심혈관 건강에 좋습니다.',
  '근력 운동은 근육을 강화하고 기초 대사량을 높입니다.',
  '요가는 유연성을 향상시키고 정신적인 안정감을 줍니다.',
  '사무실에서 할 수 있는 간단한 목 스트레칭이 있습니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 1', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'section': 'Section 0', 'source': 'Exercise_Guide'},
  {'section': 'Section 1', 'source': 'Exercise_Guide'},
  {'section': 'Section 2', 'source': 'Exercise_Guide'},
  {'section': 'Section 3', 'source': 'Exercise_Guide'},
  {'section': 'Section 4', 'source': 'Exercise_Guide'}

In [30]:
chroma_db2 = Chroma(
    collection_name = "ai_sample_collection",
    persist_directory="./chroma_db",
    embedding_function=embeddings_model
)

chroma_db2.get()

{'ids': ['doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_ex_1',
  'doc_ex_2',
  'doc_ex_3',
  'doc_ex_4',
  'doc_ex_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로 머신러닝과 딥러닝을 포함합니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '스트레칭은 운동 전후에 근육을 이완시키는 데 도움이 됩니다.',
  '유산소 운동은 심혈관 건강에 좋습니다.',
  '근력 운동은 근육을 강화하고 기초 대사량을 높입니다.',
  '요가는 유연성을 향상시키고 정신적인 안정감을 줍니다.',
  '사무실에서 할 수 있는 간단한 목 스트레칭이 있습니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 1', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'section': 'Section 0', 'source': 'Exercise_Guide'},
  {'section': 'Section 1', 'source': 'Exercise_Guide'},
  {'section': 'Section 2', 'source': 'Exercise_Guide'},
  {'section': 'Section 3', 'source': 'Exercise_Guide'},
  {'section': 'Section 4', 'source': 'Exercise_Guide'}